In [ ]:
import json
import time
import re
import csv
import subprocess
from collections import Counter

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer

from razdel import tokenize as razdel_tokenize
import spacy
import pandas as pd
from sentence_transformers import SentenceTransformer, util


d:\VSCode\NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Инициализация библиотек и моделей

def ensure_spacy_model():
    # Загружает или устанавливает ru_core_news_sm
    try:
        nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])
        print("Модель spaCy ru_core_news_sm загружена")
        return nlp
    except OSError:
        print("Модель ru_core_news_sm не найдена. Устанавливаю...")
        try:
            subprocess.run(["python", "-m", "spacy", "download", "ru_core_news_sm"], check=True)
            nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])
            print("Модель успешно установлена")
            return nlp
        except Exception as e:
            print("Ошибка при установке spaCy:", e)
            return None


def ensure_pymorphy():
    # Инициализация pymorphy2 с фиксом для Python 3.13
    import inspect
    if not hasattr(inspect, "getargspec"):
        from collections import namedtuple
        ArgSpec = namedtuple("ArgSpec", "args varargs keywords defaults")
        def getargspec(func):
            spec = inspect.getfullargspec(func)
            return ArgSpec(spec.args, spec.varargs, spec.varkw, spec.defaults)
        inspect.getargspec = getargspec

    try:
        from pymorphy2 import MorphAnalyzer
        morph = MorphAnalyzer()
        print("pymorphy2 успешно инициализирован")
        return morph
    except Exception as e:
        print("Ошибка pymorphy2:", e)
        return None


def ensure_nltk_resources():
    # Проверяет и загружает нужные данные NLTK
    try:
        nltk.data.find("tokenizers/punkt_tab")
    except LookupError:
        nltk.download("punkt_tab")

ensure_nltk_resources()
spacy_nlp = ensure_spacy_model()
morph = ensure_pymorphy()
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

porter = PorterStemmer()
snowball = SnowballStemmer("russian")


✅ Модель spaCy ru_core_news_sm загружена


d:\VSCode\NLP\.venv\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ pymorphy2 успешно инициализирован


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
# Методы токенизации и нормализации

def naive_tokenize(text):
    return [t for t in text.split() if t.strip()]

def regex_tokenize(text):
    return [t for t in re.findall(r"\b\w+\b", text.lower()) if t.strip()]

def nltk_tokenize(text):
    try:
        return [t for t in word_tokenize(text, language="russian") if t.strip()]
    except:
        return []

def razdel_tokenize_text(text):
    return [t.text for t in razdel_tokenize(text) if t.text.strip()]

def spacy_tokenize(text):
    if spacy_nlp is None:
        return []
    doc = spacy_nlp(text)
    return [token.text for token in doc if token.text.strip()]

# Нормализация

def porter_stem(tokens):
    return [porter.stem(t) for t in tokens]

def snowball_stem(tokens):
    return [snowball.stem(t) for t in tokens]

def pymorphy_lemmatize(tokens):
    if morph is None:
        return tokens
    return [morph.parse(t)[0].normal_form for t in tokens]

def spacy_lemmatize(tokens):
    if spacy_nlp is None:
        return tokens
    doc = spacy_nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]


In [ ]:
# Метрики эффективности

def compute_oov(tokens_list, vocab):
    total_tokens = sum(len(toks) for toks in tokens_list)
    oov = sum(1 for toks in tokens_list for t in toks if t not in vocab)
    return oov / total_tokens * 100 if total_tokens > 0 else 0

def compute_cosine_similarity(original_text, processed_tokens):
    processed_text = " ".join(processed_tokens)
    if not processed_text or not original_text:
        return 0.0
    emb = sentence_model.encode([original_text, processed_text], convert_to_tensor=True)
    return util.cos_sim(emb[0], emb[1]).item()


In [ ]:
# Основная логика

def process_corpus(input_file="3_news_corpus_universal.jsonl"):
    texts = []
    with open(input_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                article = json.loads(line.strip())
                text = article.get("preprocessed_text", article.get("cleaned_text", article.get("text", "")))
                if text:
                    texts.append(text)
            except:
                continue
    return texts


def run_experiment(texts):
    methods = [
        ("naive", naive_tokenize, None),
        ("regex", regex_tokenize, None),
        ("nltk", nltk_tokenize, None),
        ("razdel", razdel_tokenize_text, None),
        ("nltk_porter", nltk_tokenize, porter_stem),
        ("nltk_snowball", nltk_tokenize, snowball_stem)
    ]
    if spacy_nlp:
        methods.extend([
            ("spacy", spacy_tokenize, None),
            ("spacy_lem", spacy_tokenize, spacy_lemmatize)
        ])
    if morph:
        methods.append(("nltk_pymorphy", nltk_tokenize, pymorphy_lemmatize))

    results, vocab = [], set()

    for name, tokenize_fn, normalize_fn in methods:
        print(f"\nМетод: {name}")
        start = time.time()
        tokens_list, similarities = [], []

        for text in texts:
            tokens = tokenize_fn(text)
            if normalize_fn:
                tokens = normalize_fn(tokens)
            tokens_list.append(tokens)
            if len(similarities) < 10:
                sim = compute_cosine_similarity(text, tokens)
                similarities.append(sim)

        vocab_size = len({t for toks in tokens_list for t in toks})
        vocab.update(t for toks in tokens_list for t in toks)

        results.append({
            "method": name,
            "vocab_size": vocab_size,
            "avg_similarity": sum(similarities)/len(similarities) if similarities else 0.0,
            "time_per_1000": (time.time() - start) / len(texts) * 1000,
            "oov_percentage": compute_oov(tokens_list, vocab)
        })
    return results


In [ ]:
# Сохранение результатов

def save_results(results):
    pd.DataFrame(results).to_csv("tokenization_metrics.csv", index=False)
    print("Метрики сохранены в tokenization_metrics.csv")

def save_report(results):
    md = "# Отчёт по сравнению методов токенизации и нормализации\n\n"
    md += "| Метод | Размер словаря | OOV (%) | Косинусное сходство | Время на 1000 статей (с) |\n"
    md += "|-------|----------------|---------|---------------------|--------------------------|\n"
    for r in results:
        md += f"| {r['method']} | {r['vocab_size']} | {r['oov_percentage']:.2f} | {r['avg_similarity']:.4f} | {r['time_per_1000']:.2f} |\n"
    with open("analysis_report.md", "w", encoding="utf-8") as f:
        f.write(md)
    print("📘 Отчёт сохранён в analysis_report.md")


In [ ]:
# Запуск эксперимента

texts = process_corpus("3_news_corpus_universal.jsonl")
print(f"Загружено {len(texts)} документов")

results = run_experiment(texts)

save_results(results)
save_report(results)

pd.DataFrame(results)


Загружено 1000 документов

▶️ Метод: naive

▶️ Метод: regex

▶️ Метод: nltk

▶️ Метод: razdel

▶️ Метод: nltk_porter

▶️ Метод: nltk_snowball

▶️ Метод: spacy

▶️ Метод: spacy_lem

▶️ Метод: nltk_pymorphy
📁 Метрики сохранены в tokenization_metrics.csv
📘 Отчёт сохранён в analysis_report.md


,method,vocab_size,avg_similarity,time_per_1000,oov_percentage
0,naive,44333,1.000000,1.848534,0.0
1,regex,34259,0.917866,1.177768,0.0
2,nltk,34769,0.971512,2.211550,0.0
3,razdel,34478,0.971297,1.786527,0.0
4,nltk_porter,34750,0.971005,3.552431,0.0
5,nltk_snowball,17434,0.859068,5.028917,0.0
6,spacy,34695,0.970112,50.009183,0.0
7,spacy_lem,19468,0.924172,97.268040,0.0
8,nltk_pymorphy,17603,0.929640,18.611283,0.0
